# 텍스트 전처리
1. 특수 문자 처리 등 간단한 전처리
2. 문장 교정
2. POS tag 활용하기
3. 불용어 제거

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 한국어 토큰화를 위해 konlpy를 설치합니다
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy

In [3]:
# 라이브러리 로드
import pandas as pd
import re
from konlpy.tag import Komoran

## 01 간단한 전처리

In [172]:
# 데이터 오픈
train_path = "/content/drive/MyDrive/05_강의자료/210719_SDS/nsmc_train.csv"
test_path = "/content/drive/MyDrive/05_강의자료/210719_SDS/nsmc_test.csv"

In [173]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [174]:
train.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [175]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [176]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [177]:
train = train.dropna().loc[:1000]
test = test.dropna().loc[:499]

In [178]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001 entries, 0 to 1000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1001 non-null   int64 
 1   document  1001 non-null   object
 2   label     1001 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 31.3+ KB


In [179]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        500 non-null    int64 
 1   document  500 non-null    object
 2   label     500 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 15.6+ KB


#### 본격적인 전처리를 하겠습니다.
1. 특수 문자 제거
2. 숫자 제거
3. 공백 제거

In [180]:
# 예시 문장입니다
review = " 으아아~~~ 너무 재미있느 ㄴㄴㄴㄴ 영화였다*^_^*,,,,,별점 9999"

In [181]:
# 특수 문자만 제거하겠습니다
re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\^]', '', review)

' 으아아 너무 재미있느 ㄴㄴㄴㄴ 영화였다별점 9999'

In [182]:
# 한글, 영어, 숫자빼고 다 제거하겠습니다
review = re.sub('[^가-힣a-zA-Z0-9\s]', '', review)
review

' 으아아 너무 재미있느  영화였다별점 9999'

In [183]:
# 연속 공백도 제거 해보겠습니다
review = re.sub(r'\s+', ' ', review)
review

' 으아아 너무 재미있느 영화였다별점 9999'

In [184]:
# 숫자도 제거해볼까요?
review = re.sub(r'\d+', '', review)
review

' 으아아 너무 재미있느 영화였다별점 '

In [185]:
# 특수 문자를 제거하는 파이프라인을 만들겠습니다.
def remove_punct(documents):
    cleaned_sents = []
    for sentence in documents:
        # 한글, 영어, 숫자빼고 제거
        sentence = re.sub('[^가-힣a-zA-Z0-9\s]', '', str(sentence))
        # 2개 이상의 공백 제거
        sentence = re.sub(r'\s+', ' ', sentence)
        # 시작이 공백인 경우 제거
        sentence = re.sub(r'^\s+', '', sentence)
        # 마지막이 공백인 경우 제거
        sentence = re.sub(r'\s+$', '', sentence)
        cleaned_sents.append(sentence)
    return cleaned_sents

In [186]:
train['document'] = remove_punct(train['document'])
test['document'] = remove_punct(test['document'])

In [187]:
train.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


## 02 띄어쓰기 교정

In [188]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-nexwxapc
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-nexwxapc
  Created wheel for pykospacing: filename=pykospacing-0.5-cp37-none-any.whl size=2255829 sha256=baf13f7c0997f615ac2c7453828041f4bb98b5bba334a683e7f44372830608cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-gxr09gwz/wheels/4d/45/58/e26cb2b7f6a063d234158c6fd1e5700f6e15b99d67154340ba
Successfully built pykospacing


In [189]:
from pykospacing import Spacing

In [190]:
spacing = Spacing()

In [191]:
sent = train.loc[2, 'document']
print(sent)

너무재밓었다그래서보는것을추천한다


In [192]:
spacing.get_spaced_sent(sent)

'너무 재밓었다 그래서 보는 것을 추천한다 '

In [193]:
def space_check(documents):
    checked_sentences = []
    for sentence in documents:
        checked_sentences.append(spacing.get_spaced_sent(sentence))
    return checked_sentences

In [194]:
# 1분 넘게 걸립니다..
train['document'] = space_check(train['document'])
# test['document'] = space_check(test['document'])

In [195]:
train.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터 보고 초딩 영화 줄오버 연기조차 가볍지 않구나,1
2,10265843,너무 재밓었다 그래서 보는 것을 추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬 페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커스...,1


In [196]:
train = train.drop(972)
train.index = range(len(train))

# test = test.drop([116, 254])
# test.index = range(len(train))

In [197]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1000 non-null   int64 
 1   document  1000 non-null   object
 2   label     1000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 23.6+ KB


## 02 POS tag 활용하기

In [198]:
# Komoran 객체 정의
komoran = Komoran()

In [199]:
review = train['document'][0]

In [200]:
review

'아 더빙 진짜 짜증나네요 목소리 '

In [201]:
komoran.pos(review)

[('아', 'IC'),
 ('더빙', 'NNP'),
 ('진짜', 'MAG'),
 ('짜증', 'NNG'),
 ('나', 'VV'),
 ('네요', 'EC'),
 ('목소리', 'NNG')]

한글 형태소 품사를 참고하여 중요 품사만 필터링하겠습니다.  
http://kkma.snu.ac.kr/documents/?doc=postag

In [202]:
significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA']

In [203]:
def pos_filtering(documents):
    '''
    입력 문장: 더빙 진짜 짜증나네요 목소리 
    출력 문장: 더빙/NNP 진짜/MAG 짜증/NNG 나/VV 목소리/NNG
    '''
    filtered_docu = []
    for sent in documents:
        filtered_sent = ""
        for token, tag in komoran.pos(sent):
            if tag in significant_tags:
                # filtered_sent += token + '/' + tag + ' '
                filtered_sent += token + ' '
        filtered_docu.append(filtered_sent.strip())
    return filtered_docu

In [204]:
filtered_docu = []
for idx, sent in enumerate(test['document']):
    try:
        komoran.pos(sent)
    except:
        print(idx, sent)

In [205]:
train['document'] = pos_filtering(train['document'])
test['document'] = pos_filtering(test['document'])

In [206]:
train.head()

,id,document,label
0,9976970,더빙 진짜 짜증 나 목소리,0
1,3819312,흠 포스터 보 초딩 영화 줄 오버 연기 가볍 않,1
2,10265843,너무 그래서 보 것 추천 하,0
3,9045019,교도소 이야기 솔직히 재미 없 평점 조정,0
4,6483659,사이몬 익살 스럽 연기 돋보이 영화 스파이더맨 늙 보이 하 커스틴 던스트 너무나,1


## 03 Stopwords
* 이미 POS tag를 활용하는 단계에서 조사같은 불용어가 삭제되었습니다.
* 하지만, 각 도메인에서의 불용어 집합이 다를 수 있기 때문에, 추가적으로 불용어를 정의해서 삭제하는 방법을 알아봅니다.

In [207]:
# 불용어 정의
stopwords = ['흠/NNG', '좀/MAG', '수/NNB', '등/NNB', '데/NNB']

In [208]:
def remove_stopwords(documents):
    removed_sentences = []
    for sent in documents:
        modi_sent = []
        for word in sent.split(' '):
            if word not in stopwords:
                modi_sent.append(word)
        removed_sentences.append(' '.join(modi_sent))
    return removed_sentences

In [209]:
train['document'] = remove_stopwords(train['document'])
test['document'] = remove_stopwords(test['document'])

In [210]:
train.head()

,id,document,label
0,9976970,더빙 진짜 짜증 나 목소리,0
1,3819312,흠 포스터 보 초딩 영화 줄 오버 연기 가볍 않,1
2,10265843,너무 그래서 보 것 추천 하,0
3,9045019,교도소 이야기 솔직히 재미 없 평점 조정,0
4,6483659,사이몬 익살 스럽 연기 돋보이 영화 스파이더맨 늙 보이 하 커스틴 던스트 너무나,1


In [211]:
train.to_csv("/content/drive/MyDrive/05_강의자료/210719_SDS/nsmc_train_preprocessed.csv")
test.to_csv("/content/drive/MyDrive/05_강의자료/210719_SDS/nsmc_test_preprocessed.csv")